<a href="https://colab.research.google.com/github/novoforce/Exploring-Tensorflow/blob/main/GANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist,mnist

((images_train,_),(_,_)) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#normalizing the data between -1 to +1 --> (to compensate for the noise in the range)
images_train = (np.array(images_train).astype(np.float32) - 127.5)/127.5  #(normalizing between -1 to 1)
images_train = images_train.reshape(60000,784)

# Generator Architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import initializers
randomDim= 100
adam= Adam(learning_rate=0.0002,beta_1=0.5)
generator= Sequential()
generator.add(Dense(256,input_dim=randomDim,kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh')) # since original image is between -1 to +1 so tanh will give -1 to +1
generator.compile(loss='binary_crossentropy',optimizer=adam)

In [ ]:
generator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 784)              

# Discriminator architecture

In [ ]:
from tensorflow.keras.layers import Dropout
discriminator= Sequential()
discriminator.add(Dense(1024,input_dim=784,kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid')) # probability of being real(1) or fake(0)
discriminator.compile(loss='binary_crossentropy',optimizer=adam)

In [ ]:
discriminator.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)              

# Connecting the architectures

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
ganInput= Input(shape=(randomDim,)) #100
x= generator(ganInput) #784
ganOutput= discriminator(x) #1
gan= Model(inputs=ganInput,outputs=ganOutput)
gan.compile(loss="binary_crossentropy",optimizer=adam)
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_3 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 2,946,577
Non-trainable params: 0
_________________________________________________________________
